## PyTorch Introduction 

In [1]:
import torch
import numpy as np
from torch import nn 


> How to create a Neural Network ?

In [2]:
class LineNetowrk(nn.Module):
    def __init__(self):
        super().__init__()
        # Starting the neuron
        self.layers = nn.Sequential(
            nn.Linear(1,1)
        )

    def forward(self,x): 
        return self.layers(x)
        

SyntaxError: incomplete input (152698812.py, line 2)

> How to train a Neural Network ?

> Creating a database

In [ ]:
from torch.utils.data import Dataset, dataloader
import torch.distributions.uniform as urandom

class AlgebraicDataset(Dataset):
    def __init__(self, f, interval, nsamples):
        X = urandom.Uniform(interval[0], interval[1]).sample([nsamples])
        self.data = [(x,f(x)) for x in X]

    def __len__(self):
        return len(self.data) 

    def __getitem__(self,idx):
        return self.data[idx]   


> Create Training Dataset  

In [ ]:
linear = lambda x: 2*x + 3  
interval = (-10, 10)
train_n_samples = 1000 
test_n_samples = 100

In [ ]:
train_dataset  = AlgebraicDataset(linear,interval,train_n_samples)
test_dataset  = AlgebraicDataset(linear,interval,test_n_samples)

train_Dataloader = dataloader(train_dataset,batch_size=train_n_samples,shuffle=True)
test_Dataloader = dataloader(train_dataset,batch_size=test_n_samples,shuffle=True)

> Hyperparameter 

In [ ]:
# Loss function 
# Mean Squred error
lossfunc = nn.MSELoss()